# Manage Qiskit Serverless compute and data resources

Qiskit Serverless allows you to manage compute and data across your [Qiskit pattern](./intro-to-patterns), including CPUs, QPUs, and other compute accelerators.

## Parallel workflows

For classical tasks that can be parallelized, use the `@distribute_task` decorater to define compute requirements needed to perform a task. Start by recalling the `transpile_parallel.py` example from the [Write your first Qiskit Serverless program](./serverless-first-program) topic:

In [1]:
# This cell is hidden from users
from pathlib import Path
Path("./source_files").mkdir(exist_ok=True, parents=True)

In [2]:
%%writefile ./source_files/transpile_remote.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_serverless import distribute_task

service = QiskitRuntimeService()

@distribute_task(target={"cpu": 1})
def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit (or list of circuits) into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
		backend=service.backend(backend)
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

Writing ./source_files/transpile_remote.py


In this example, you decorated the `transpile_remote()` method with `@distribute_task(target={"cpu": 1})`. When run, this creates an asynchronous parallel worker task with a single CPU core, and returns with a reference to track the worker. To fetch the result, pass the reference to the `get()` method:

In [3]:
%%writefile --append ./source_files/transpile_remote.py

from qiskit_serverless import get, get_arguments, save_result

arguments = get_arguments()
circuit = arguments.get("circuit")
optimization_level = arguments.get("optimization_level")
backend = arguments.get("backend")

transpile_worker_reference = transpile_remote(
    circuit,
    optimization_level,
    backend
)
result = get(transpile_worker_reference)
save_result(result)

Appending to ./source_files/transpile_remote.py


In [4]:
%%writefile --append ./source_files/transpile_remote.py
# This cell is hidden from users. It checks the transpilation ran correctly.
from qiskit import QuantumCircuit
assert isinstance(result, QuantumCircuit)

Appending to ./source_files/transpile_remote.py


You can also create and run multiple parallel tasks as follows:

In [5]:
%%writefile --append ./source_files/transpile_remote.py

transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend)
    for circuit in arguments.get("circuit_list")
]

results = get(transpile_worker_references)
save_result(results)  # Overwrites any previously saved results

Appending to ./source_files/transpile_remote.py


In [6]:
%%writefile --append ./source_files/transpile_remote.py
# This cell is hidden from users. It checks the transpilation ran correctly.
assert isinstance(results[0], QuantumCircuit)

Appending to ./source_files/transpile_remote.py


In [7]:
# This cell is hidden from users.
# It uploads the serverless program and checks it runs.

from qiskit.circuit.random import random_circuit
from qiskit_serverless import IBMServerlessClient, QiskitFunction
from IPython.display import clear_output
import time

TITLE = "transpile_remote_serverless_test"  # TODO: Randomize names to avoid conflict
serverless = IBMServerlessClient()
transpile_remote_demo = QiskitFunction(
    title=TITLE,
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
	dependencies=["qiskit-aer==0.14.1"],
)
serverless.upload(transpile_remote_demo)
job = serverless.get(TITLE).run(
    circuit=random_circuit(3,3),
    circuit_list=[random_circuit(3,3) for _ in range(3)],
    backend="ibm_kyiv",
    optimization_level=1,
)
for retry in range(20):
    status = job.status()
    if status == "DONE":
        break
    if status == "ERROR":
        raise Exception("Something went wrong. See following serverless logs:\n\n" + job.logs())
    clear_output()
    print(f"Job still running, waiting 5s then retrying ({retry})")
    time.sleep(5)

### Explore different task configurations

You can flexibly allocate CPU, GPU, and memory for your tasks via `@distribute_task()`. For Qiskit Serverless on IBM Quantum&trade; Platform, each program is equipped with 16 CPU cores and 32 GB RAM, which can be allocated dynamically as needed.

CPU cores can be allocated as full CPU cores, or even fractional allocations, as shown in the following.

Memory is allocated in number of bytes. Recall that there are 1024 bytes in a kilobyte, 1024 kilobytes in a megabyte, and 1024 megabytes in a gigabyte. To allocate 2 GB of memory for your worker, you need to allocate `"mem": 2 * 1024 * 1024 * 1024`.

In [8]:
%%writefile ./source_files/transpile_remote.py

@distribute_task(target={
    "cpu": 16,
    "mem": 32 * 1024 * 1024 * 1024
})
def transpile_remote(circuit, optimization_level, backend):
    return None

Overwriting ./source_files/transpile_remote.py


In [9]:
# This cell is hidden from users
# Clean up file afterwards
Path("./source_files/transpile_remote.py").unlink()

## Automatic QPU selection

This example demonstrates how to use `IBMQPUSelector` to automate the process of selecting which qubits to use from a set of available QPUs. Instead of manually selecting a QPU, Qiskit Serverless automatically allocates a QPU according to desired criteria. `IBMQPUSelector`s are imported from server-side `qiskit_serverless_tools.selectors` modules, and must be invoked from your uploaded program.

Here, `IBMLeastNoisyQPUSelector` finds the QPU that yields the least-noisy qubit subgraph for the input circuit, from among the QPUs available to you through your IBM Quantum account.

For each `IBMQPUSelector`, the context is set in the constructor. All `IBMQPUSelectors` require Qiskit Runtime credentials. The `IBMLeastNoisyQPUSelector` requires a circuit and transpile options specifying how the circuit should be optimized for each QPU, to determine the most optimal QPU and qubit layout.

All `IBMQPUSelectors` implement a `get_backend` method, which retrieves the optimal QPU with respect to the given context. The `get_backend` method also allows for additional filtering of the QPUs. It is implemented using the same interface as the [QiskitRuntimeService.backends](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backends) method.

In [10]:
%%writefile ./source_files/transpile_parallel.py

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.random import random_circuit
from qiskit_serverless_tools.selectors import IBMLeastNoisyQPUSelector

service = QiskitRuntimeService()

abstract_circuit = random_circuit(
    num_qubits=100, depth=4, measure=True
)

selector = IBMLeastNoisyQPUSelector(
    service, circuit=abstract_circuit, transpile_options={"optimization_level": 3}
)
backend = selector.get_backend(min_num_qubits=127)
target_circuit = selector.optimized_circuit

Overwriting ./source_files/transpile_parallel.py


You can also use the `IBMLeastBusyQPUSelector` to find a QPU that can support the circuit width but with the shortest queue.

In [11]:
%%writefile --append ./source_files/transpile_parallel.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_serverless_tools.selectors import IBMLeastBusyQPUSelector

backend = IBMLeastBusyQPUSelector(service).get_backend(min_num_qubits=127)
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
target_circuit = pm.run(abstract_circuit)

Appending to ./source_files/transpile_parallel.py


## Manage data across your program

Qiskit Serverless allows you to manage files in the `/data` directory across all your programs. This includes several limitations:

- Only `tar` and `h5` files are supported today
- This is only a flat `/data` storage, and cannot have `/data/folder/` subdirectories

The following shows how to upload files. Be sure you have authenticated to Qiskit Serverless with your [IBM Quantum account](https://quantum.ibm.com/account) (see [Deploy to IBM Quantum Platform](./serverless-first-program#deploy-to-ibm-quantum-platform) for instructions).

In [12]:
import tarfile

# Create a tar
filename = "transpile_demo.tar"
file = tarfile.open(filename,"w")
file.add("./source_files/transpile_parallel.py")
file.close()

# Upload the tar to Serverless data directory
serverless.file_upload(filename)

'{"message":"/usr/src/app/media/5e1f442128cdf60018496a04/transpile_demo.tar"}'

Next, you can list all the files in your `data` directory. This data is accessible to all programs.

In [13]:
serverless.files()

['transpile_demo.tar']

This can be done from a program by using `file_download()` to download the file to the program environment, and uncompressing the `tar`.

In [14]:
%%writefile --append ./source_files/transpile_parallel.py

import tarfile

files = serverless.files()
demo_file = files[0]
downloaded_tar = serverless.file_download(demo_file)

print(downloaded_tar)
with tarfile.open(downloaded_tar, 'r') as tar:
    tar.extractall()

Appending to ./source_files/transpile_parallel.py


In [15]:
# This cell is hidden from users.
# It uploads the serverless program and checks it runs.

from qiskit.circuit.random import random_circuit
from qiskit_serverless import IBMServerlessClient, QiskitFunction
import time

TITLE = "transpile_remote_serverless_test"  # TODO: Randomize names to avoid conflict
serverless = IBMServerlessClient()
transpile_remote_demo = QiskitFunction(
    title=TITLE,
    entrypoint="transpile_parallel.py",
    working_dir="./source_files/",
	dependencies=["qiskit-aer==0.14.1"],
)
serverless.upload(transpile_remote_demo)
job = serverless.get(TITLE).run(
    circuit=random_circuit(3,3),
    circuit_list=[random_circuit(3,3) for _ in range(3)],
    backend="ibm_kyiv",
    optimization_level=1,
)
for retry in range(20):
    status = job.status()
    if status == "DONE":
        break
    if status == "ERROR":
        raise Exception("Something went wrong. See following serverless logs:\n\n" + job.logs())
    clear_output()
    print(f"Job still running, waiting 5s then retrying ({retry})")
    time.sleep(5)

Exception: Something went wrong. See following serverless logs:

2024-09-13 15:49:52,472	INFO job_manager.py:531 -- Runtime env is setting up.
Traceback (most recent call last):
  File "/tmp/ray/session_2024-09-13_15-49-42_152664_1/runtime_resources/working_dir_files/_ray_pkg_f23b08bce382b975/transpile_parallel.py", line 4, in <module>
    from qiskit_serverless_tools.selectors import IBMLeastNoisyQPUSelector
ModuleNotFoundError: No module named 'qiskit_serverless_tools'


At this point, your program can interact with the files, as you would a local experiment. `file_upload()` , `file_download()`, and `file_delete()` can be called from your local experiment, or your uploaded program, for consistent and flexible data management.

## Next steps

<Admonition type="info" title="Recommendations">

- See a full example that [ports existing code to Qiskit Serverless ](./serverless-port-code).
- Read a paper in which researchers used Qiskit Serverless and quantum-centric supercomputing to [explore quantum chemistry](https://arxiv.org/abs/2405.05068v1).

</Admonition>